In [ ]:
!pip install numpy
!pip install pandas
!pip install re
!pip install transformers
!pip install stopwords
!pip install torch
!pip install torchtext

In [ ]:
!pip install pandas torch transformers nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
rutaarchivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"
rutaarchivo2 = "/content/drive/MyDrive/Investigación/economic_dictionary.xlsx"
DataEnt = pd.read_excel(rutaarchivo2)
DataEnt.head()

##BERT

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch

def entrenar_modelo_bert(data, text, sentiment, num_epochs=3):
    # Cargar el modelo BERT pre-entrenado y el tokenizador
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    # Dividir los datos en entrenamiento y prueba
    X = data[text]
    y = data[sentiment]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Tokenizar los textos y convertirlos en tensores
    train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors="pt", max_length=512)
    test_encodings = tokenizer(list(X_test), truncation=True, padding=True, return_tensors="pt", max_length=512)

    # Crear tensores para las etiquetas
    train_labels = torch.tensor(list(y_train), dtype=torch.long)
    test_labels = torch.tensor(list(y_test), dtype=torch.long)

    # Entrenar el modelo BERT
    train_data = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
    test_data = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=8)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=8)

    device = torch.device("cuda" if torch.cuda.is available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    # Guardar el modelo entrenado
    model.save_pretrained("modelo_entrenado_bert")

    return "Entrenamiento completado"

# Cargar los datos de los tweets desde un archivo Excel
rutaarchivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx" # Reemplaza con la ruta real de tu archivo Excel
tweets = pd.read_excel(rutaarchivo)

# Entrenar el modelo BERT
resultado_entrenamiento = entrenar_modelo_bert(tweets, "text", "sentiment")
print(resultado_entrenamiento)




In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

def cargar_modelo_y_predecir(textos_nuevos, modelo_entrenado):
    # Cargar el modelo BERT pre-entrenado y el tokenizador
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Cargar el modelo entrenado
    model = BertForSequenceClassification.from_pretrained(modelo_entrenado)

    # Tokenizar los nuevos textos
    encodings = tokenizer(list(textos_nuevos), truncation=True, padding=True, return_tensors="pt", max_length=512)

    # Realizar predicciones con el modelo
    model.eval()
    with torch.no_grad():
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)

    return predicted_labels

# Cargar los datos desde el archivo Excel "data" con una columna llamada "tweet"
data = pd.read_excel("/content/drive/MyDrive/Investigación/Data.xlsx")

# Extraer los tweets de la columna "tweet"
nuevos_textos = data["tweet"].tolist()

# Cargar el modelo entrenado y realizar predicciones
modelo_entrenado = "/content/drive/MyDrive/Investigación/modelo_entrenado_bert"  # Reemplaza con la ruta real de tu modelo entrenado
predicciones = cargar_modelo_y_predecir(nuevos_textos, modelo_entrenado)

# Escalar los resultados al rango -1 a 1
predicciones_escaladas = (predicciones.float() / 1) * 2 - 1

# Crear un DataFrame con los resultados
resultados_df = pd.DataFrame({"Texto": nuevos_textos, "Sentimiento": predicciones_escaladas.numpy()})

# Guardar los resultados en un archivo Excel
resultados_df.to_excel("/content/drive/MyDrive/Investigación/resultados_sentimiento1.xlsx", index=False)


##Redes

In [ ]:
!pip install pandas numpy scikit-learn tensorflow tensorflow-cpu spacy

In [ ]:
!python -m spacy download es_core_news_sm

In [38]:
import spacy
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Montar Google Drive
drive.mount('/content/drive')

# Ruta al archivo de datos en Google Drive
data_path = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"

# Cargar los datos
data = pd.read_excel(data_path)
X = data['text']  # Características (tweets)
y = data['sentiment']  # Etiquetas (sentimientos)

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Obtener la lista de palabras vacías en español de spaCy
stop_words_spanish = nlp.Defaults.stop_words

# Convierte la lista de palabras vacías en español a una lista simple
stop_words_spanish = list(stop_words_spanish)

# Preprocesamiento de texto
vectorizer = TfidfVectorizer(max_features=1000, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(X)
X = X.toarray()  # Convertir la matriz dispersa en una matriz NumPy densa

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir el modelo de red neuronal
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test)
print(f'Error cuadrático medio en el conjunto de prueba: {loss:.2f}')

# Utilizar el modelo para predecir sentimientos
predicted_sentiments = model.predict(X_test)

# Guardar el modelo en un archivo
model.save("/content/drive/MyDrive/Investigación/modelo_analisis_sentimientored.h5")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/10
1/1 [==============================] - 1s 649ms/step - loss: 0.7097
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6255
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 0.5538
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.4936
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.4384
Epoch 6/10
1/1 [==============================] - 0s 9ms/step - loss: 0.3881
Epoch 7/10
1/1 [==============================] - 0s 9ms/step - loss: 0.3441
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 0.3044
Epoch 9/10
1/1 [==============================] - 0s 11ms/step - loss: 0.2684
Epoch 10/10
1/1 [==============================] - 0s 110ms/step - loss: 1.3648
Error cuadrático medio en el conjunto de prueba: 1.36
1/1 [==============================] -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
import pandas as pd

# Cargar los datos
data = pd.read_excel("/content/drive/MyDrive/Investigación/Data.xlsx")

## Agregar una columna sentiment con un valor predeterminado de neutral
#data["sentiment"] = "neutral"

# Guardar los datos
data.to_excel("/content/drive/MyDrive/Investigación/Data_con_Sentiment.xlsx", index=False)


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Cargar el modelo entrenado
loaded_model = load_model("/content/drive/MyDrive/Investigación/modelo_analisis_sentimientored.h5")

# Cargar los datos de entrada
#data = pd.read_excel("/content/drive/MyDrive/Investigación/Data_con_Sentiment.xlsx")
data = pd.read_excel("/content/drive/MyDrive/Investigación/Data.xlsx")
tweets = data['text']

# Preprocesamiento de texto
X = vectorizer.transform(tweets).toarray()

# Predecir el sentimiento de los tweets
predicted_sentiments = loaded_model.predict(X)

# Agregar los puntajes de sentimiento al DataFrame original
data['sentiment_score'] = predicted_sentiments

# Guardar el DataFrame resultante en un nuevo archivo Excel
data.to_excel("/content/drive/MyDrive/Investigación/Data_con_Puntajes_con_Sentiment.xlsx", index=False)


1/1 [==============================] - 0s 114ms/step
